### Tennessee opioid risk modeling 
Normal adjacency matrix - Dimentions would be different.

{'DC': ['EP_MUNIT', selected_variables_tn[0], 'EP_MINRTY', 'EP_AGE65', 'EP_CROWD'], 'TN': [selected_variables_tn[0], 'EP_NOHSDP', 'EP_PCI', 'EP_MOBILE', 'EP_POV'], 'WY': [selected_variables_tn[0], 'EP_PCI', 'EP_LIMENG', 'EP_CROWD', 'EP_UNINSUR']}

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import pickle as pickle
from pylab import *
%matplotlib inline

import warnings

# Ignore FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

In [ ]:
tn_svi_nod = gpd.read_file('./data/processed data/SVI with HepVu census tracts/SVI2018 TN census tracts with death rate HepVu/SVI2018_TN_census_tracts_with_death_rate_HepVu.shp')

##### Filtering the raw variables with overdose death rate

In [ ]:
selected_variables = ['EP_POV','EP_UNEMP','EP_PCI','EP_NOHSDP','EP_UNINSUR','EP_AGE65','EP_AGE17','EP_DISABL','EP_SNGPNT','EP_LIMENG','EP_MINRTY','EP_MUNIT','EP_MOBILE','EP_CROWD','EP_NOVEH','EP_GROUPQ','NOD_Rate']

In [ ]:
selected_variables_without_y = ['EP_POV','EP_UNEMP','EP_PCI','EP_NOHSDP','EP_UNINSUR','EP_AGE65','EP_AGE17','EP_DISABL','EP_SNGPNT','EP_LIMENG','EP_MINRTY','EP_MUNIT','EP_MOBILE','EP_CROWD','EP_NOVEH','EP_GROUPQ']

In [ ]:
selected_variables_tn_with_geo = ['FIPS','EP_DISABL', 'EP_NOHSDP', 'EP_PCI', 'EP_MOBILE', 'EP_POV','NOD_Rate','geometry']

In [ ]:
selected_variables_tn = ['EP_DISABL', 'EP_NOHSDP', 'EP_PCI', 'EP_MOBILE', 'EP_POV']

In [ ]:
selected_variables_tn_with_od = ['EP_DISABL', 'EP_NOHSDP', 'EP_PCI', 'EP_MOBILE', 'EP_POV','NOD_Rate']

In [ ]:
# get the number of rows with 999 or -999 values in the selected_variablesWy_ columns
rows_with_n999 = tn_svi_nod[(tn_svi_nod['EP_DISABL'] == -999) | (tn_svi_nod['EP_NOHSDP'] == -999) | (tn_svi_nod['EP_PCI'] == -999) | (tn_svi_nod['EP_MOBILE'] == -999) | (tn_svi_nod['EP_POV'] == -999)].shape[0]

In [ ]:
rows_with_n999

In [ ]:
# if value equals -999, replace with 0 in selected_variablesWy_
tn_svi_nod[selected_variables_tn[0]] = tn_svi_nod[selected_variables_tn[0]].replace(-999, 0)
tn_svi_nod[selected_variables_tn[1]] = tn_svi_nod[selected_variables_tn[1]].replace(-999, 0)
tn_svi_nod[selected_variables_tn[2]] = tn_svi_nod[selected_variables_tn[2]].replace(-999, 0)
tn_svi_nod[selected_variables_tn[3]] = tn_svi_nod[selected_variables_tn[3]].replace(-999, 0)
tn_svi_nod[selected_variables_tn[4]] = tn_svi_nod[selected_variables_tn[4]].replace(-999, 0)

In [ ]:
tn_svi_nod[selected_variables_tn_with_geo]

#### Check the distribution of each selected variable

In [ ]:
tn_svi_nod[selected_variables_tn].describe()

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(15, 10))
for i, ax in zip(selected_variables_tn, axes.flatten()):
    tn_svi_nod[i].plot.hist(bins=20, alpha=0.5, ax=ax, edgecolor='black')
    ax.set_title(i, fontsize=12)  # Adjust the fontsize here

plt.tight_layout()
plt.show()


In [ ]:
# calculate the 50th and 75th and 90th percentile for each variable
variable_1_percentile_50 = tn_svi_nod[selected_variables_tn[0]].quantile(0.5)
variable_1_percentile_75 = tn_svi_nod[selected_variables_tn[0]].quantile(0.75)
variable_1_percentile_90 = tn_svi_nod[selected_variables_tn[0]].quantile(0.9)

variable_2_percentile_50 = tn_svi_nod[selected_variables_tn[1]].quantile(0.5)
variable_2_percentile_75 = tn_svi_nod[selected_variables_tn[1]].quantile(0.75)
variable_2_percentile_90 = tn_svi_nod[selected_variables_tn[1]].quantile(0.9)

variable_3_percentile_50 = tn_svi_nod[selected_variables_tn[2]].quantile(0.5)
variable_3_percentile_75 = tn_svi_nod[selected_variables_tn[2]].quantile(0.75)
variable_3_percentile_90 = tn_svi_nod[selected_variables_tn[2]].quantile(0.9)

variable_4_percentile_50 = tn_svi_nod[selected_variables_tn[3]].quantile(0.5)
variable_4_percentile_75 = tn_svi_nod[selected_variables_tn[3]].quantile(0.75)
variable_4_percentile_90 = tn_svi_nod[selected_variables_tn[3]].quantile(0.9)

variable_5_percentile_50 = tn_svi_nod[selected_variables_tn[4]].quantile(0.5)
variable_5_percentile_75 = tn_svi_nod[selected_variables_tn[4]].quantile(0.75)
variable_5_percentile_90 = tn_svi_nod[selected_variables_tn[4]].quantile(0.9)

In [ ]:
# for each selected vcariable wy calculate the number of rows that are below 50th, 75th and 90th percentile
variable_1_below_50 = tn_svi_nod[tn_svi_nod[selected_variables_tn[0]] < variable_1_percentile_50].shape[0]
variable_1_below_75 = tn_svi_nod[tn_svi_nod[selected_variables_tn[0]] < variable_1_percentile_75].shape[0]
variable_1_below_90 = tn_svi_nod[tn_svi_nod[selected_variables_tn[0]] < variable_1_percentile_90].shape[0]

variable_2_below_50 = tn_svi_nod[tn_svi_nod[selected_variables_tn[1]] < variable_2_percentile_50].shape[0]
variable_2_below_75 = tn_svi_nod[tn_svi_nod[selected_variables_tn[1]] < variable_2_percentile_75].shape[0]
variable_2_below_90 = tn_svi_nod[tn_svi_nod[selected_variables_tn[1]] < variable_2_percentile_90].shape[0]

variable_3_below_50 = tn_svi_nod[tn_svi_nod[selected_variables_tn[2]] < variable_3_percentile_50].shape[0]
variable_3_below_75 = tn_svi_nod[tn_svi_nod[selected_variables_tn[2]] < variable_3_percentile_75].shape[0]
variable_3_below_90 = tn_svi_nod[tn_svi_nod[selected_variables_tn[2]] < variable_3_percentile_90].shape[0]

variable_4_below_50 = tn_svi_nod[tn_svi_nod[selected_variables_tn[3]] < variable_4_percentile_50].shape[0]
variable_4_below_75 = tn_svi_nod[tn_svi_nod[selected_variables_tn[3]] < variable_4_percentile_75].shape[0]
variable_4_below_90 = tn_svi_nod[tn_svi_nod[selected_variables_tn[3]] < variable_4_percentile_90].shape[0]

variable_5_below_50 = tn_svi_nod[tn_svi_nod[selected_variables_tn[4]] < variable_5_percentile_50].shape[0]
variable_5_below_75 = tn_svi_nod[tn_svi_nod[selected_variables_tn[4]] < variable_5_percentile_75].shape[0]
variable_5_below_90 = tn_svi_nod[tn_svi_nod[selected_variables_tn[4]] < variable_5_percentile_90].shape[0]

# create a dataframe with the results
data = {'Variable': selected_variables_tn, '50th Percentile': [variable_1_percentile_50, variable_2_percentile_50, variable_3_percentile_50, variable_4_percentile_50, variable_5_percentile_50], '75th Percentile': [variable_1_percentile_75, variable_2_percentile_75, variable_3_percentile_75, variable_4_percentile_75, variable_5_percentile_75], '90th Percentile': [variable_1_percentile_90, variable_2_percentile_90, variable_3_percentile_90, variable_4_percentile_90, variable_5_percentile_90], 'Rows below 50th Percentile': [variable_1_below_50, variable_2_below_50, variable_3_below_50, variable_4_below_50, variable_5_below_50], 'Rows below 75th Percentile': [variable_1_below_75, variable_2_below_75, variable_3_below_75, variable_4_below_75, variable_5_below_75], 'Rows below 90th Percentile': [variable_1_below_90, variable_2_below_90, variable_3_below_90, variable_4_below_90, variable_5_below_90]}
percentile_df = pd.DataFrame(data)

percentile_df

#### Get number of rows below and above mean

In [ ]:
# EP_DISABL
variable_1_mean = tn_svi_nod[selected_variables_tn[0]].mean()
variable_1_sd = tn_svi_nod[selected_variables_tn[0]].std()

# EP_PCI 
variable_2_mean = tn_svi_nod[selected_variables_tn[1]].mean()
variable_2_sd = tn_svi_nod[selected_variables_tn[1]].std()

# EP_LIMENG
variable_3_mean = tn_svi_nod[selected_variables_tn[2]].mean()
variable_3_sd = tn_svi_nod[selected_variables_tn[2]].std()

# EP_CROWD
variable_4_mean = tn_svi_nod[selected_variables_tn[2]].mean()
variable_4_sd = tn_svi_nod[selected_variables_tn[2]].std()

# EP_UNINSUR
variable_5_mean = tn_svi_nod[selected_variables_tn[3]].mean()
variable_5_sd = tn_svi_nod[selected_variables_tn[3]].std()



In [ ]:
# for each selected vcariable wy calculate the number of rows that are within 1, 2 standard deviations from the mean
# EP_DISABL
variable_1_1sd = tn_svi_nod[(tn_svi_nod[selected_variables_tn[0]] >= variable_1_mean - variable_1_sd) & (tn_svi_nod[selected_variables_tn[0]] <= variable_1_mean + variable_1_sd)].shape[0]
variable_1_2sd = tn_svi_nod[(tn_svi_nod[selected_variables_tn[0]] >= variable_1_mean - 2*variable_1_sd) & (tn_svi_nod[selected_variables_tn[0]] <= variable_1_mean + 2*variable_1_sd)].shape[0]

# EP_PCI
variable_2_1sd = tn_svi_nod[(tn_svi_nod[selected_variables_tn[1]] >= variable_2_mean - variable_2_sd) & (tn_svi_nod[selected_variables_tn[1]] <= variable_2_mean + variable_2_sd)].shape[0]
variable_2_2sd = tn_svi_nod[(tn_svi_nod[selected_variables_tn[1]] >= variable_2_mean - 2*variable_2_sd) & (tn_svi_nod[selected_variables_tn[1]] <= variable_2_mean + 2*variable_2_sd)].shape[0]

# EP_LIMENG
variable_3_1sd = tn_svi_nod[(tn_svi_nod[selected_variables_tn[2]] >= variable_3_mean - variable_3_sd) & (tn_svi_nod[selected_variables_tn[2]] <= variable_3_mean + variable_3_sd)].shape[0]
variable_3_2sd = tn_svi_nod[(tn_svi_nod[selected_variables_tn[2]] >= variable_3_mean - 2*variable_3_sd) & (tn_svi_nod[selected_variables_tn[2]] <= variable_3_mean + 2*variable_3_sd)].shape[0]

# EP_CROWD
variable_4_1sd = tn_svi_nod[(tn_svi_nod[selected_variables_tn[3]] >= variable_4_mean - variable_4_sd) & (tn_svi_nod[selected_variables_tn[3]] <= variable_4_mean + variable_4_sd)].shape[0]
variable_4_2sd = tn_svi_nod[(tn_svi_nod[selected_variables_tn[3]] >= variable_4_mean - 2*variable_4_sd) & (tn_svi_nod[selected_variables_tn[3]] <= variable_4_mean + 2*variable_4_sd)].shape[0]

# EP_UNINSUR
variable_5_1sd = tn_svi_nod[(tn_svi_nod[selected_variables_tn[4]] >= variable_5_mean - variable_5_sd) & (tn_svi_nod[selected_variables_tn[4]] <= variable_5_mean + variable_5_sd)].shape[0]
variable_5_2sd = tn_svi_nod[(tn_svi_nod[selected_variables_tn[4]] >= variable_5_mean - 2*variable_5_sd) & (tn_svi_nod[selected_variables_tn[4]] <= variable_5_mean + 2*variable_5_sd)].shape[0]

# create a dataframe with the results
data = {'Variable': [selected_variables_tn[0], 'EP_PCI', selected_variables_tn[2], selected_variables_tn[3], selected_variables_tn[4]], 'Mean': [variable_1_mean, variable_2_mean, variable_3_mean, variable_4_mean, variable_5_mean], 'Standard Deviation': [variable_1_sd, variable_2_sd, variable_3_sd, variable_4_sd, variable_5_sd], 'Within 1 SD': [variable_1_1sd, variable_2_1sd, variable_3_1sd, variable_4_1sd, variable_5_1sd], 'Within 2 SD': [variable_1_2sd, variable_2_2sd, variable_3_2sd, variable_4_2sd, variable_5_2sd]}
df = pd.DataFrame(data)
df

In [ ]:
# for each selected vcariable wy calculate the number of rows that are below 1 standard deviation from the mean and rows below mean and above 1 standard deviation
# EP_DISABL
variable_1_below_1sd = tn_svi_nod[(tn_svi_nod[selected_variables_tn[0]] < variable_1_mean - variable_1_sd)].shape[0]
variable_1_below_mean = tn_svi_nod[(tn_svi_nod[selected_variables_tn[0]] < variable_1_mean)].shape[0]
variable_1_below_mean_plus_1sd = tn_svi_nod[(tn_svi_nod[selected_variables_tn[0]] < variable_1_mean + variable_1_sd)].shape[0]

# EP_PCI
variable_2_below_1sd = tn_svi_nod[(tn_svi_nod[selected_variables_tn[1]] < variable_2_mean - variable_2_sd)].shape[0]
variable_2_below_mean = tn_svi_nod[(tn_svi_nod[selected_variables_tn[1]] < variable_2_mean)].shape[0]
variable_2_below_mean_plus_1sd = tn_svi_nod[(tn_svi_nod[selected_variables_tn[1]] < variable_2_mean + variable_2_sd)].shape[0]

# EP_LIMENG
variable_3_below_1sd = tn_svi_nod[(tn_svi_nod[selected_variables_tn[2]] < variable_3_mean - variable_3_sd)].shape[0]
variable_3_below_mean = tn_svi_nod[(tn_svi_nod[selected_variables_tn[2]] < variable_3_mean)].shape[0]
variable_3_below_mean_plus_1sd = tn_svi_nod[(tn_svi_nod[selected_variables_tn[2]] < variable_3_mean + variable_3_sd)].shape[0]

# EP_CROWD
variable_4_below_1sd = tn_svi_nod[(tn_svi_nod[selected_variables_tn[3]] < variable_4_mean - variable_4_sd)].shape[0]
variable_4_below_mean = tn_svi_nod[(tn_svi_nod[selected_variables_tn[3]] < variable_4_mean)].shape[0]
variable_4_below_mean_plus_1sd = tn_svi_nod[(tn_svi_nod[selected_variables_tn[3]] < variable_4_mean + variable_4_sd)].shape[0]

# EP_UNINSUR
variable_5_below_1sd = tn_svi_nod[(tn_svi_nod[selected_variables_tn[4]] < variable_5_mean - variable_5_sd)].shape[0]
variable_5_below_mean = tn_svi_nod[(tn_svi_nod[selected_variables_tn[4]] < variable_5_mean)].shape[0]
variable_5_below_mean_plus_1sd = tn_svi_nod[(tn_svi_nod[selected_variables_tn[4]] < variable_5_mean + variable_5_sd)].shape[0]

# create a dataframe with the results
data = {'Variable': [selected_variables_tn[0], selected_variables_tn[1], selected_variables_tn[2], selected_variables_tn[3], selected_variables_tn[4]], 'Below Mean-1 SD': [variable_1_below_1sd, variable_2_below_1sd, variable_3_below_1sd, variable_4_below_1sd, variable_5_below_1sd], 'Below Mean': [variable_1_below_mean, variable_2_below_mean, variable_3_below_mean, variable_4_below_mean, variable_5_below_mean], 'Below Mean + 1 SD': [variable_1_below_mean_plus_1sd, variable_2_below_mean_plus_1sd, variable_3_below_mean_plus_1sd, variable_4_below_mean_plus_1sd, variable_5_below_mean_plus_1sd]}
df = pd.DataFrame(data)
df

In [ ]:
fig, ax = plt.subplots()
tn_svi_nod[selected_variables_tn[0]].plot.hist(bins=20, alpha=0.8, ax=ax, edgecolor='black')

# Axis formatting.
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_color('#DDDDDD')
ax.tick_params(bottom=False, left=False)
ax.set_axisbelow(True)
ax.yaxis.grid(True, color='#EEEEEE')
ax.xaxis.grid(False)

# Add text annotations to the top of the bars.
for bar in ax.patches:
    bar_value = int(bar.get_height())
    text = f'{bar_value:,}'
    text_x = bar.get_x() + bar.get_width() / 2
    text_y = bar.get_y() + bar_value
    bar_color = bar.get_facecolor()
    ax.text(text_x, text_y, text, ha='center', va='bottom', color=ax.patches[0].get_facecolor(),
            size=8)
    
ax.axvline(tn_svi_nod[selected_variables_tn[0]].mean() - tn_svi_nod[selected_variables_tn[0]].std(), color='r', linestyle='dashed', linewidth=1)
ax.text(tn_svi_nod[selected_variables_tn[0]].mean() - tn_svi_nod[selected_variables_tn[0]].std(), ax.get_ylim()[1], 'Mean-1Sd', color='r', va='bottom', ha='right')


# mean
ax.axvline(tn_svi_nod[selected_variables_tn[0]].mean(), color='r', linestyle='dashed', linewidth=1)
ax.text(tn_svi_nod[selected_variables_tn[0]].mean(), ax.get_ylim()[1], 'Mean', color='r', va='bottom', ha='right')


ax.axvline(tn_svi_nod[selected_variables_tn[0]].mean() + tn_svi_nod[selected_variables_tn[0]].std(), color='r', linestyle='dashed', linewidth=1)
ax.text(tn_svi_nod[selected_variables_tn[0]].mean() + 2*tn_svi_nod[selected_variables_tn[0]].std(), ax.get_ylim()[1], 'Mean+1Sd', color='r', va='bottom', ha='right')


# axhline
ax.hlines(y=2.6, xmin=0, xmax=tn_svi_nod[selected_variables_tn[0]].mean()+ tn_svi_nod[selected_variables_tn[0]].std(), color='g', linestyle='dashed', linewidth=2)
ax.hlines(y=2.4, xmin=0, xmax=tn_svi_nod[selected_variables_tn[0]].mean(), color='g', linestyle='dashed', linewidth=2)
ax.hlines(y=2.2, xmin=0, xmax=tn_svi_nod[selected_variables_tn[0]].mean()- tn_svi_nod[selected_variables_tn[0]].std(), color='g', linestyle='dashed', linewidth=2)


    
# Add labels and a title.
ax.set_xlabel('Estimated Value', labelpad=15, color='#333333')
ax.set_ylabel('Frequency', labelpad=15, color='#333333')
ax.set_title('Disability population distribution', pad=15, color='#333333')

fig.tight_layout()
plt.show()

### Adjacency Method

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import itertools
from itertools import combinations
from scipy import spatial
import pickle as pickle
import gudhi
from pylab import *
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
import io
from tqdm import tqdm
from PIL import Image, ImageDraw, ImageChops, ImageFont
import shapely.geometry as geom
from shapely.ops import unary_union
import warnings

import invr

In [ ]:
tn_filtered = tn_svi_nod[selected_variables_tn_with_geo]

In [ ]:
#reset index
tn_filtered = tn_filtered.reset_index(drop=True)

In [ ]:
tn_filtered.head(3)

In [ ]:
# get the uniques fips codes
fips = tn_filtered['FIPS'].unique()
fips

In [ ]:
# # select the rows where the variable is below the 50th percentile
# variable_1_below_50 = tn_filtered[tn_filtered[selected_variables_tn[1]] < variable_2_percentile_75]

# # get the shape
# variable_1_below_50.shape

Generate adjacent counties

In [ ]:
def generate_adjacent_counties(dataframe,filtration_threshold,variable_name):

    
    filtered_df = dataframe[dataframe[variable_name] < filtration_threshold]

    # Perform a spatial join to find adjacent precincts
    adjacent_counties = gpd.sjoin(filtered_df, filtered_df, predicate='intersects', how='left')

    # Filter the results to include only the adjacent states
    adjacent_counties = adjacent_counties.query('sortedID_left != sortedID_right')

    # Group the resulting dataframe by the original precinct Name and create a list of adjacent precinct Name
    adjacent_counties = adjacent_counties.groupby('sortedID_left')['sortedID_right'].apply(list).reset_index()

    adjacent_counties.rename(columns={'sortedID_left': 'county', 'sortedID_right': 'adjacent'}, inplace=True)

    adjacencies_list = adjacent_counties['adjacent'].tolist()
    county_list = adjacent_counties['county'].tolist()

    merged_df = pd.merge(adjacent_counties, dataframe, left_on='county',right_on='sortedID', how='left')
    merged_df = gpd.GeoDataFrame(merged_df, geometry='geometry')

    return adjacencies_list,merged_df,county_list

In [ ]:
def form_simplicial_complex(adjacent_county_list,county_list):
    max_dimension = 3

    V = []
    V = invr.incremental_vr(V, adjacent_county_list, max_dimension,county_list)

    return V

In [ ]:
def fig2img(fig):
     #convert matplot fig to image and return it

     buf = io.BytesIO()
     fig.savefig(buf)
     buf.seek(0)
     img = Image.open(buf)
     return img

In [ ]:
def plot_simplicial_complex(dataframe,V):

    #city centroids
    city_coordinates = {city.sortedID: np.array((city.geometry.centroid.x, city.geometry.centroid.y)) for _, city in dataframe.iterrows()}

    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.set_axis_off() 

    # Plot the "wyoming_svi" DataFrame
    dataframe.plot(ax=ax, edgecolor='black', linewidth=0.3, color="white")

    # Plot the centroid of the large square with values
    # for i, row in dataframe.iterrows():
    #     centroid = row['geometry'].centroid
    #     # text_to_display = f"FIPS: {row['FIPS']}\nFilteration: {row['EP_SNGPNT']}"
    #     plt.text(centroid.x, centroid.y, str(row['FIPS']), fontsize=8, ha='center', color="black")
    #     # plt.text(centroid.x, centroid.y, text_to_display, fontsize=10, ha='center', color="black")

    for edge_or_traingle in V:

        
        if len(edge_or_traingle) == 2:
            # Plot an edge
            ax.plot(*zip(*[city_coordinates[vertex] for vertex in edge_or_traingle]), color='red', linewidth=1)
            # img = fig2img(fig)
            # list_gif.append(img)
        elif len(edge_or_traingle) == 3:
            # Plot a triangle
            ax.add_patch(plt.Polygon([city_coordinates[vertex] for vertex in edge_or_traingle], color='green', alpha=0.2))
            # img = fig2img(fig)
            # list_gif.append(img)
    plt.show()
    plt.close()

    #return list_gif #deleted  plot_simplicial_complex(dataframe,V,list_gif) --> list part

##### Multiple variable code starts here

In [ ]:
selected_variables_and_threshold = {selected_variables_tn[0]: variable_1_percentile_50, selected_variables_tn[1]: variable_2_percentile_50, selected_variables_tn[2]: variable_3_percentile_50, selected_variables_tn[3]: variable_4_percentile_50, selected_variables_tn[4]: variable_5_percentile_50}

In [ ]:
# create a empty dictionary
edges_and_traingles_for_each_variable_below_50th_percentile = {}

In [ ]:
for variable_name, threshold in selected_variables_and_threshold.items():

    # Sorting based on the variable and selecting only the FIPS and the variable columns is important
    # Also we need to keep  the dataframe sorted based on the variable

    df_one_variable = tn_filtered[['FIPS',variable_name, 'geometry']]

    # # Sorting the DataFrame based on the 'rate' column
    df_one_variable = df_one_variable.sort_values(by=variable_name)
    df_one_variable['sortedID'] = range(len(df_one_variable))

    # Convert the DataFrame to a GeoDataFrame
    df_one_variable = gpd.GeoDataFrame(df_one_variable, geometry='geometry')
    df_one_variable.crs = "EPSG:3395"  # This is a commonly used projected CRS


    # print(df_one_variable.head(100))

    adjacencies_list,adjacent_counties_df,county_list = generate_adjacent_counties(df_one_variable,threshold,variable_name)

    # create a dictionary adjacent_counties_df column county as key and column adjacent as value(to avoid NULL adjacencies error)
    adjacent_counties_dict = dict(zip(adjacent_counties_df['county'],adjacent_counties_df['adjacent']))

    # this take only counties that have adjacent counties
    county_list = adjacent_counties_df['county'].tolist()

    V = form_simplicial_complex(adjacent_counties_dict,county_list)

    # This is a new feature that I added to the code. It creates a new list replace the sorted ID with the FIPS on the V list
    # create a new list replace the sorted ID with the FIPS on the V list
    V_FIPS = [[df_one_variable.iloc[x]['FIPS'] for x in i] for i in V]
    
    #add V list to the edges_and_traingles_for_each_variable dictionary with the key as the variable name
    edges_and_traingles_for_each_variable_below_50th_percentile[variable_name] = V_FIPS

    # # # store the list of images for each variable
    # # list_img = []

    # # plot the simplicial complex
    print(f"Plotting simplicial complex for {variable_name} variable at threshold {threshold}")
    plot_simplicial_complex(df_one_variable,V)

In [ ]:
selected_variables_and_threshold = {selected_variables_tn[0]: variable_1_percentile_75, selected_variables_tn[1]: variable_2_percentile_75, selected_variables_tn[2]: variable_3_percentile_75, selected_variables_tn[3]: variable_4_percentile_75, selected_variables_tn[4]: variable_5_percentile_75}
# create a empty dictionary
edges_and_traingles_for_each_variable_below_75th_percentile = {}

In [ ]:
for variable_name, threshold in selected_variables_and_threshold.items():

    # Sorting based on the variable and selecting only the FIPS and the variable columns is important
    # Also we need to keep  the dataframe sorted based on the variable

    df_one_variable = tn_filtered[['FIPS',variable_name, 'geometry']]

    # # Sorting the DataFrame based on the 'rate' column
    df_one_variable = df_one_variable.sort_values(by=variable_name)
    df_one_variable['sortedID'] = range(len(df_one_variable))

    # Convert the DataFrame to a GeoDataFrame
    df_one_variable = gpd.GeoDataFrame(df_one_variable, geometry='geometry')
    df_one_variable.crs = "EPSG:3395"  # This is a commonly used projected CRS

    adjacencies_list,adjacent_counties_df,county_list = generate_adjacent_counties(df_one_variable,threshold,variable_name)

    # create a dictionary adjacent_counties_df column county as key and column adjacent as value(to avoid NULL adjacencies error)
    adjacent_counties_dict = dict(zip(adjacent_counties_df['county'],adjacent_counties_df['adjacent']))

    # this take only counties that have adjacent counties
    county_list = adjacent_counties_df['county'].tolist()

    V = form_simplicial_complex(adjacent_counties_dict,county_list)

    # This is a new feature that I added to the code. It creates a new list replace the sorted ID with the FIPS on the V list
    # create a new list replace the sorted ID with the FIPS on the V list
    V_FIPS = [[df_one_variable.iloc[x]['FIPS'] for x in i] for i in V]

    #add V list to the edges_and_traingles_for_each_variable dictionary with the key as the variable name
    edges_and_traingles_for_each_variable_below_75th_percentile[variable_name] = V_FIPS

    # # # store the list of images for each variable
    # # list_img = []

    # # plot the simplicial complex
    print(f"Plotting simplicial complex for {variable_name} variable at threshold {threshold}")
    plot_simplicial_complex(df_one_variable,V)

In [ ]:
selected_variables_and_threshold = {selected_variables_tn[0]: variable_1_percentile_90, selected_variables_tn[1]: variable_2_percentile_90, selected_variables_tn[2]: variable_3_percentile_90, selected_variables_tn[3]: variable_4_percentile_90, selected_variables_tn[4]: variable_5_percentile_90}
# create a empty dictionary
edges_and_traingles_for_each_variable_below_90th_percentile = {}

In [ ]:
for variable_name, threshold in selected_variables_and_threshold.items():

    # Sorting based on the variable and selecting only the FIPS and the variable columns is important
    # Also we need to keep  the dataframe sorted based on the variable

    df_one_variable = tn_filtered[['FIPS',variable_name, 'geometry']]

    # # Sorting the DataFrame based on the 'rate' column
    df_one_variable = df_one_variable.sort_values(by=variable_name)
    df_one_variable['sortedID'] = range(len(df_one_variable))

    # Convert the DataFrame to a GeoDataFrame
    df_one_variable = gpd.GeoDataFrame(df_one_variable, geometry='geometry')
    df_one_variable.crs = "EPSG:3395"  # This is a commonly used projected CRS

    adjacencies_list,adjacent_counties_df,county_list = generate_adjacent_counties(df_one_variable,threshold,variable_name)

    # create a dictionary adjacent_counties_df column county as key and column adjacent as value(to avoid NULL adjacencies error)
    adjacent_counties_dict = dict(zip(adjacent_counties_df['county'],adjacent_counties_df['adjacent']))

    # this take only counties that have adjacent counties
    county_list = adjacent_counties_df['county'].tolist()

    V = form_simplicial_complex(adjacent_counties_dict,county_list)

    # This is a new feature that I added to the code. It creates a new list replace the sorted ID with the FIPS on the V list
    # create a new list replace the sorted ID with the FIPS on the V list
    V_FIPS = [[df_one_variable.iloc[x]['FIPS'] for x in i] for i in V]

    #add V list to the edges_and_traingles_for_each_variable dictionary with the key as the variable name
    edges_and_traingles_for_each_variable_below_90th_percentile[variable_name] = V_FIPS

    # # # store the list of images for each variable
    # # list_img = []

    # # plot the simplicial complex
    print(f"Plotting simplicial complex for {variable_name} variable at threshold {threshold}")
    plot_simplicial_complex(df_one_variable,V)

##### Create a matrix for the full region(without filtering)

In [ ]:
selected_regions_variable_1 = []
selected_regions_variable_2 = []
selected_regions_variable_3 = []
selected_regions_variable_4 = []
selected_regions_variable_5 = []

This loop decides which percentile we are selecting

In [ ]:
# loop through the dictionary and for each variable create a list of edges
for variable_name, V_FIPS in edges_and_traingles_for_each_variable_below_75th_percentile.items():
# for variable_name, V_FIPS in edges_and_traingles_for_each_variable_below_mean_plus_1sd.items():
    for set in V_FIPS:
        if len(set) == 2 or len(set) == 3:
            # if variable is EP_DISABL
            if variable_name == selected_variables_tn[0]:
                #check if the edge(both values) is not already in the list
                for vertice in set:
                    if vertice not in selected_regions_variable_1:
                        selected_regions_variable_1.append(vertice)
            elif variable_name == selected_variables_tn[1]:
                for vertice in set:
                    if vertice not in selected_regions_variable_2:
                        selected_regions_variable_2.append(vertice)
            elif variable_name == selected_variables_tn[2]:
                for vertice in set:
                    if vertice not in selected_regions_variable_3:
                        selected_regions_variable_3.append(vertice)
            elif variable_name == selected_variables_tn[3]:
                for vertice in set:
                    if vertice not in selected_regions_variable_4:
                        selected_regions_variable_4.append(vertice)
            elif variable_name == selected_variables_tn[4]:
                for vertice in set:
                    if vertice not in selected_regions_variable_5:
                        selected_regions_variable_5.append(vertice)
        


In [ ]:
# this set of variables needed - specially STCNTY to identify the county
selected_variables_tn_with_censusinfo = ['FIPS','STCNTY','EP_DISABL', 'EP_NOHSDP', 'EP_PCI', 'EP_MOBILE', 'EP_POV','NOD_Rate','geometry']

In [ ]:
# get a filtered dataframe with the selected regions
filtered_df_census = tn_svi_nod[selected_variables_tn_with_censusinfo]

In [ ]:
# get the unique counties
unique_counties = tn_svi_nod['STCNTY'].unique()

In [ ]:
len(unique_counties)

Saving selected countoes as a okl file fir all the variables(dictionary)

In [ ]:
# loop through the unique counties and get the number of rows for each county
i=0
for county in unique_counties:
    print(f"County: {county}")

    # create a temp_df with the selected county
    temp_census = filtered_df_census[filtered_df_census['STCNTY'] == county]

    # filter the filtered_df_census dataframe to only include the selected census for each selected variable
    # FIPS includes the census info
    variable_1_selected_census = temp_census[temp_census['FIPS'].isin(selected_regions_variable_1)][['FIPS',selected_variables_tn[0],'geometry']]
    variable_2_selected_census = temp_census[temp_census['FIPS'].isin(selected_regions_variable_2)][['FIPS',selected_variables_tn[1],'geometry']]
    variable_3_selected_census = temp_census[temp_census['FIPS'].isin(selected_regions_variable_3)][['FIPS',selected_variables_tn[2],'geometry']]
    variable_4_selected_census = temp_census[temp_census['FIPS'].isin(selected_regions_variable_4)][['FIPS',selected_variables_tn[3],'geometry']]
    variable_5_selected_census = temp_census[temp_census['FIPS'].isin(selected_regions_variable_5)][['FIPS',selected_variables_tn[4],'geometry']]

    # create a new column in df that contains the x and y coordinates of the centroid of each polygon
    variable_1_selected_census['coords'] = variable_1_selected_census['geometry'].apply(lambda x: x.representative_point().coords[:])
    variable_1_selected_census['coords'] = [coords[0] for coords in variable_1_selected_census['coords']]

    variable_2_selected_census['coords'] = variable_2_selected_census['geometry'].apply(lambda x: x.representative_point().coords[:])
    variable_2_selected_census['coords'] = [coords[0] for coords in variable_2_selected_census['coords']]

    variable_3_selected_census['coords'] = variable_3_selected_census['geometry'].apply(lambda x: x.representative_point().coords[:])
    variable_3_selected_census['coords'] = [coords[0] for coords in variable_3_selected_census['coords']]

    variable_4_selected_census['coords'] = variable_4_selected_census['geometry'].apply(lambda x: x.representative_point().coords[:])
    variable_4_selected_census['coords'] = [coords[0] for coords in variable_4_selected_census['coords']]

    variable_5_selected_census['coords'] = variable_5_selected_census['geometry'].apply(lambda x: x.representative_point().coords[:])
    variable_5_selected_census['coords'] = [coords[0] for coords in variable_5_selected_census['coords']]


    # # plotting the centroids
    # plt.scatter(*zip(*variable_1_selected_census['coords']), s=10, c='r')
    # plt.show()

    # plt.scatter(*zip(*variable_2_selected_census['coords']), s=10, c='r')
    # plt.show()

    # plt.scatter(*zip(*variable_3_selected_census['coords']), s=10, c='r')
    # plt.show()

    # plt.scatter(*zip(*variable_4_selected_census['coords']), s=10, c='r')
    # plt.show()

    # plt.scatter(*zip(*variable_5_selected_census['coords']), s=10, c='r')
    # plt.show()

    # create a dictionary with variable name as key and the data as value for all the selected regions
    selected_coordinates_dic = {selected_variables_tn[0]: variable_1_selected_census, selected_variables_tn[1]: variable_2_selected_census, selected_variables_tn[2]: variable_3_selected_census, selected_variables_tn[3]: variable_4_selected_census, selected_variables_tn[4]: variable_5_selected_census}


    # save the dictionary to a pickle file
    with open(f'./results/selected coordinates for each county - percentiles/selected_coordinates_dic_{county}.pkl', 'wb') as f:
        pickle.dump(selected_coordinates_dic, f)

    
